In [1]:
import requests
import json
import numpy as np
import cvxopt.solvers

In [2]:
class server_api:
    def __init__(self, user='', pwd=''):
        self.url = 'https://dolphin.jump-technology.com:3389/api/v1/'
        self.s = requests.Session()
        self.s.auth = (user, pwd)
        self.s.headers.update({'x-test': 'true'})
    def expand_params(self, req, params):
        request = ''
        if type(params) is tuple:
            request = self.url + (req.format(*params))
        else:
            request = self.url + (req.format(params))
        return request
    def get(self, req, params):
        request = self.expand_params(req, params)
        self.res = ''
        try:
            self.data = self.s.get(request, headers={'x-test2': 'true'}, verify=False)
            self.res = json.loads(self.data.text)
        except:
            print('Error while connecting')
        return self.res
    def post(self, req, params, body):
        request = self.expand_params(req, params)
        self.res = ''
        try:
            self.data = self.s.post(url=request, json=body, headers={'x-test2': 'true'}, verify=False)
            self.res = json.loads(self.data.text)
        except Exception as err:
            print(err)
            print('Error while connecting')
        return self.res
    def get_cached(self):
        return self.res

In [3]:
server = server_api('epita_user_7', 'dolphin19423')

In [4]:
server.post('ratio/invoke', (), {
'_ratio':[17, 20, 22, 18],
'_asset':[564],
'_bench':None,
'_startDate':'2015-01-01',
'_endDate':'2017-01-01',
'_frequency':None})

C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{}

In [116]:
class portfolio_manager:
    def __init__(self, date_begin, date_end, nb_assets=20, beg=1):
        self.assets = []
        off = 0
        i = beg
        self.names = []
        while i < nb_assets + off + beg:
            asset = server.get('asset/{}/quote?start_date={}&end_date={}', (i + 1, date_begin, date_end))
            if type(asset) is list:
                self.assets.append(np.array(asset))
            else:
                while type(asset) != list:
                    #print(type(asset))
                    j = i + 1
                    asset = server.get('asset/{}/quote?start_date={}&end_date={}', (j + 1, date_begin, date_end))
                    j += 1
                off += (j - i)
                i = j
                #print('end else ', i, j, off)
                self.assets.append(np.array(asset))
            data = server.get('asset/{}', (i + 1))
            self.names.append(data['LABEL']['value'])
            print('Adding asset', i, 'to the portfolio')
            i += 1
        self.assets = np.array(self.assets)

    def get_assets(self):
        return self.assets

    def compute_means_and_cov(self):
        self.assets_means = []
        rets = []

        min_size = 9999999999
        for asset in range(self.assets.shape[0]):
            #print(self.assets[asset].shape)
            returns = [self.assets[asset][i]['return'] for i in range(self.assets[asset].shape[0])]
            min_size = min(min_size, len(returns))
            rets.append(np.array(returns))
            mean_ret = np.mean(returns)
            self.assets_means.append(mean_ret)
    
        rets = np.array(rets)

        #print(min_size)

        self.rets_norm = []
        for asset in range(self.assets.shape[0]):
            self.rets_norm.append(rets[asset][:min_size])
    
        self.rets_norm = np.array(self.rets_norm)
        self.cov = np.cov(self.rets_norm)
        
        #print(self.cov.shape)
        
        return (np.array(self.assets_means), self.cov)
    
    def find_optimal_portfolio(self, risk_tolerance = 1.0):
        means, cov = p_m.compute_means_and_cov()
        
        m = means.shape[0]
        print('Optimizing', m, 'assets')
        #print(m)
        r_min = 0.1
        
        risk_tolerance = 1.0
        #print((-0.5 * risk_tolerance * means).shape)
        q = cvxopt.matrix(-0.5 * risk_tolerance * means)
        
        P = cvxopt.matrix(cov)

        A = cvxopt.matrix(np.ones(m), (1, m), 'd')
        b = cvxopt.matrix(np.ones(1))
        
        G = cvxopt.matrix(-np.identity(m))
        h = cvxopt.matrix(np.zeros(m))
        
        s = cvxopt.solvers.qp(P, q, G, h, A, b)['x']
        return s

In [117]:
p_m = portfolio_manager('2012-01-01', '2017-06-01', 5)

C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


Adding asset 3 to the portfolio


C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


Adding asset 4 to the portfolio


C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


Adding asset 5 to the portfolio


C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


Adding asset 6 to the portfolio


C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
C:\Users\cpcdoy\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


Adding asset 7 to the portfolio


In [98]:
s = p_m.find_optimal_portfolio()
print(s)
print(np.sum(s))

Optimizing  5  assets
     pcost       dcost       gap    pres   dres
 0: -1.6983e-04 -1.0005e+00  1e+00  1e-16  3e+00
 1: -1.6993e-04 -1.0470e-02  1e-02  1e-16  3e-02
 2: -1.7887e-04 -5.6618e-04  4e-04  5e-17  1e-03
 3: -3.3231e-04 -4.5226e-04  1e-04  2e-16  3e-19
 4: -3.5869e-04 -3.6879e-04  1e-05  2e-16  2e-19
 5: -3.6451e-04 -3.6512e-04  6e-07  1e-17  2e-19
 6: -3.6492e-04 -3.6494e-04  1e-08  1e-16  1e-19
Optimal solution found.
[ 9.27e-01]
[ 1.81e-06]
[ 3.00e-04]
[ 7.25e-02]
[ 6.97e-06]

1.0
